[Reference](https://towardsdatascience.com/end-to-end-automl-train-and-serve-with-h2o-mlflow-fastapi-and-streamlit-5d36eedfe606)

In [2]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import opendatasets as od
import os
# Retrieve data directly from source (using Kaggle API credentials, found in kaggle.json)
od.download("https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction",
           './data/raw')

raw_path = './data/raw/health-insurance-cross-sell-prediction/'
os.listdir(raw_path)

Skipping, found downloaded files in "./data/raw/health-insurance-cross-sell-prediction" (use force=True to force download)


['sample_submission.csv', 'test.csv', 'train.csv']

In [6]:
pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.2 MB 65 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.1-py2.py3-none-any.whl size=177276478 sha256=d629f7b45b38743ead5bd9657f41a61b765b22825a902e0f82388c6de5b293c2
  Stored in directory: /root/.cache/pip/wheels/a1/a1/d9/bb37df368c4635a707e7362d1088450b606041f05aeba5f173
Successfully built h2o


In [7]:
import h2o

# Initiate H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.16" 2022-07-19; OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04); OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpv5k9i0iv
  JVM stdout: /tmp/tmpv5k9i0iv/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpv5k9i0iv/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_vk9uda
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [11]:
!pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.9 MB 22.1 MB/s 
     |████████████████████████████████| 147 kB 75.2 MB/s 
     |████████████████████████████████| 79 kB 11.2 MB/s 
     |████████████████████████████████| 209 kB 96.5 MB/s 
     |████████████████████████████████| 182 kB 83.9 MB/s 
     |████████████████████████████████| 77 kB 9.3 MB/s 
     |████████████████████████████████| 78 kB 9.8 MB/s 
     |████████████████████████████████| 140 kB 90.7 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 59 kB 8.4 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139102 sha256=dc36d93df5612f65aa5855e88685f16167725ab836ba50143a9c2c4a87c6e2f7
  Stored in directory: /root/.cache/pip/wheels/3f

In [12]:
import mlflow
from mlflow import MlflowClient

# Initialize MLFlow client
client = MlflowClient()

# Set up MlFlow experiment
experiment_name = 'automl-insurance'

try:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = client.get_experiment_by_name(experiment_name)
except:
    experiment = client.get_experiment_by_name(experiment_name)
   
# Set experiment
mlflow.set_experiment(experiment_name)

# Print experiment details
print(f"Name: {experiment_name}")
print(f"Experiment_id: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")
print(f"Lifecycle_stage: {experiment.lifecycle_stage}")
print(f"Tracking uri: {mlflow.get_tracking_uri()}")

Name: automl-insurance
Experiment_id: 1
Artifact Location: file:///content/mlruns/1
Lifecycle_stage: active
Tracking uri: file:///content/mlruns


data  mlruns  sample_data


In [16]:
# Import data directly as H2O frame
main_frame = h2o.import_file(path='./data/raw/health-insurance-cross-sell-prediction/train.csv')

# Set predictor and target columns
target = 'Response'
predictors = [n for n in main_frame.col_names if n != target]

# Factorize target variable so that AutoML tackles this as classification task
main_frame[target] = main_frame[target].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
# import h2o
from h2o.automl import H2OAutoML

# Wrap autoML training with MLflow
with mlflow.start_run():
    aml = H2OAutoML(
                    max_models=13, # Run AutoML for n base models
                    seed=42, 
                    balance_classes=True, # Target classes imbalanced, so set this as True
                    sort_metric='logloss', # Sort models by logloss (metric for multi-classification)
                    verbosity='info', # Turn on verbose info
                    exclude_algos = ['GLM', 'DRF'], # Specify algorithms to exclude
                )

In [20]:
# Initiate AutoML training
aml.train(x=predictors, y=target, training_frame=main_frame)

AutoML progress: |
13:09:34.241: Project: AutoML_1_20221009_130934
13:09:34.243: 5-fold cross-validation will be used.
13:09:34.262: Setting stopping tolerance adaptively based on the training frame: 0.001619852227945515
13:09:34.262: Build control seed: 42
13:09:34.263: training frame: Frame key: AutoML_1_20221009_130934_training_py_1_sid_ad9a    cols: 12    rows: 381109  chunks: 8    size: 4882368  checksum: -9194789481094117677
13:09:34.263: validation frame: NULL
13:09:34.263: leaderboard frame: NULL
13:09:34.263: blending frame: NULL
13:09:34.263: response column: Response
13:09:34.263: fold column: null
13:09:34.263: weights column: null
13:09:34.331: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealing (7g, 10w)]}

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_1_20221009_130934

No summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.08208818268569192
RMSE: 0.2865103535401328
LogLoss: 0.2509893534647698
AUC: 0.8722260285707891
AUCPR: 0.40931470762943484
Gini: 0.7444520571415782
Null degrees of freedom: 9844
Residual degrees of freedom: 9835
Null deviance: 7152.921997782414
Residual deviance: 4941.980369721318
AIC: 4961.980369721318

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.27029390545029147
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      7256  1426  0.1642   (1426.0/8682.0)
1      368   795   0.3164   (368.0/1163.0)
Total  7624  2221  0.1822   (1794.0/9845.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.270294     0.469858  163
max f2                       0.150028     0.645044  261
max f0point5                 0.334988     0.414967  104
max accuracy                 0.407214     0.884713  44
max precision                0.561268     1         0
max recall                   0.00287922   1         389
max specificity              0.561268     1         0
max absolute_mcc             0.18104      0.419766  235
max min_per_class_accuracy   0.224909     0.785039  202
max mean_per_class_accuracy  0.150028     0.811565  261
max tns                      0.561268     8682      0
max fns                      0.561268     1162      0
max fps                      0.000374392  8682      399
max tps                      0.00287922   1163      389
max tnr                      0.561268     1         0
max fnr                      0.561268     0.99914   0
max fpr                      0.000374392  1         399
max tpr                      0.00287922   1         389

Gains/Lift Table: Avg response rate: 11.81 %, avg score: 12.00 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100559                   0.440003           4.87389     4.87389            0.575758         0.467074     0.575758                    0.467074            0.0490112       0.0490112                  387.389   387.389            0.0441736
2        0.0200102                   0.411235           4.57811     4.72675            0.540816         0.423502     0.558376                    0.445398            0.0455718       0.094583                   357.811   372.675            0.0845622
3        0.030066                    0.398411           3.67679     4.37558            0.434343         0.404176     0.516892                    0.431611            0.0369733       0.131556                   267.679   337.558            0.115085
4        0.0400203                   0.387251           3.97345     4.27556            0.469388         0.392503     0.505076                    0.421884            0.0395529       0.171109                   297.345   327.556            0.148649
5        0.0500762                   0.376799           3.59129     4.13815            0.424242         0.381888     0.488844                    0.413852            0.0361135       0.207223                   259.129   313.815            0.178197
6        0.100051                    0.343323           2.95937     3.54936            0.349593         0.361221     0.419289                    0.387563            0.147893        0

In [22]:
import mlflow.h2o

# Set metrics to log
mlflow.log_metric("log_loss", aml.leader.logloss())
mlflow.log_metric("AUC", aml.leader.auc())

# Log and save best model (mlflow.h2o provides API for logging & loading H2O models)
mlflow.h2o.log_model(aml.leader, artifact_path="model")

model_uri = mlflow.get_artifact_uri("model")
print(f'AutoML best model saved in {model_uri}')

# Get IDs of current experiment run
exp_id = experiment.experiment_id
run_id = mlflow.active_run().info.run_id

# Save leaderboard as CSV
lb = get_leaderboard(aml, extra_columns='ALL')
lb_path = f'mlruns/{exp_id}/{run_id}/artifacts/model/leaderboard.csv'
lb.as_data_frame().to_csv(lb_path, index=False) 

In [23]:
from mlflow.entities import ViewType

# Get best model amongst all runs in all experiments
all_exps = [exp.experiment_id for exp in client.list_experiments()]
runs = mlflow.search_runs(experiment_ids=all_exps, run_view_type=ViewType.ALL)
run_id, exp_id = runs.loc[runs['metrics.log_loss'].idxmin()]['run_id'], runs.loc[runs['metrics.log_loss'].idxmin()]['experiment_id']

# Load best model (AutoML leader)
best_model = mlflow.h2o.load_model(f"mlruns/{exp_id}/{run_id}/artifacts/model/")

# Generate predictions with best model (output is H2O frame)
preds_frame = best_model.predict(X_test_frame)

# Convert to pandas series
y_pred = preds_frame.as_data_frame()['predict']

In [24]:
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse

# Create FastAPI instance
app = FastAPI()

# Initiate H2O instance and MLflow client
h2o.init()
client = MlflowClient()

# Load best model (based on logloss) amongst all runs in all experiments
all_exps = [exp.experiment_id for exp in client.list_experiments()]
runs = mlflow.search_runs(experiment_ids=all_exps, run_view_type=ViewType.ALL)
run_id, exp_id = runs.loc[runs['metrics.log_loss'].idxmin()]['run_id'], runs.loc[runs['metrics.log_loss'].idxmin()]['experiment_id']
best_model = mlflow.h2o.load_model(f"mlruns/{exp_id}/{run_id}/artifacts/model/")

@app.post("/predict")
async def predict(file: bytes = File(...)):
    print('[+] Initiate Prediction')
    file_obj = io.BytesIO(file)
    test_df = pd.read_csv(file_obj)
    test_h2o = h2o.H2OFrame(test_df)

    # Generate predictions with best model (output is H2O frame)
    preds = best_model.predict(X_h2o)
    
    json_compatible_item_data = jsonable_encoder(preds)
    return JSONResponse(content=json_compatible_item_data)

In [ ]:
# !uvicorn main:app --host 0.0.0.0 --port 8000

In [ ]:
# import streamlit as st
# import requests
# import pandas as pd
# import io
# import json

# st.title('End-to-End AutoML Project: Life Insurance Assessment')

# # FastAPI endpoint
# endpoint = 'http://localhost:8000/predict'
# test_csv = st.file_uploader('', type=['csv','xlsx'], accept_multiple_files=False)

# if test_csv:
#     test_df = pd.read_csv(test_csv)
#     st.subheader('View Sample of Test Set')
#     st.write(test_df.head())

#     test_bytes_obj = io.BytesIO()
#     test_df.to_csv(test_bytes_obj, index=False)  # write to BytesIO buffer
#     test_bytes_obj.seek(0) # Reset pointer to avoid EmptyDataError
#     files = {"file": ('test_dataset.csv', test_bytes_obj, "multipart/form-data")}

#     if st.button('Start Prediction'):
#         with st.spinner('Prediction in Progress. Please Wait...'):
#             output = requests.post(endpoint, files=files, timeout=8000)
#         st.success('Success! Click the Download button below to retrieve prediction results (JSON format')
#         st.download_button(
#             label='Download',
#             data=json.dumps(output.json()), # Download as JSON file object
#             file_name='automl_prediction_results.json'
#         )

In [ ]:
# uvicorn main:app --host 0.0.0.0 --port 8000